In [ ]:
import airsim


class Drone:
    def __init__(self, vehicle_name: str, client: airsim.MultirotorClient, velocity=5):
        self.vehicle_name = vehicle_name
        self.client = client
        self.velocity = velocity

    def takeoff(self):
        self.client.enableApiControl(True, vehicle_name=self.vehicle_name)
        self.client.armDisarm(True, vehicle_name=self.vehicle_name)
        return self.client.takeoffAsync(vehicle_name=self.vehicle_name)

    def get_pose(self):
        return self.client.simGetVehiclePose(self.vehicle_name)

    def move_to_position(self, x, y, z):
        self.client.moveToPositionAsync(x, y, z, self.velocity, vehicle_name=self.vehicle_name).join()

    def execute_routine(self, routine):
        for x, y, z in routine:
            self.move_to_position(x, y, z)

In [ ]:
import time

client = airsim.MultirotorClient()
client.confirmConnection()

In [ ]:
print('Adding second drone')
pose = drones[0].get_pose()
pose.position.x_val += 10
if not client.simAddVehicle('drone_2', 'simpleflight', pose):
    print('Failed to add drone')
    return

In [ ]:
client.reset()
time.sleep(1)
drones = [Drone('drone_1', client), Drone('drone_2', client)]
print('Taking off')
time.sleep(1)
tasks = [drone.takeoff() for drone in drones]
for task in tasks:
    task.join()
print('Moving')
for drone in drones:
    pose = drone.get_pose()
    drone.move_to_position(pose.position.x_val + 5, pose.position.y_val, pose.position.z_val + 10)